<a href="https://colab.research.google.com/github/zuzKomar/PAD/blob/main/PAD07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scipy
!pip install statmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement statmodels (from versions: none)
ERROR: No matching distribution found for statmodels


Zadanie 1:
Przeprowadź test T studenta na danych w pliku wyniki.csv Hipoteza zerowa brzmi: nie ma istotnej różnicy w Ocenie końcowej pomiędzy uczniami i uczennicami. Hipoteza alternatywna brzmi: istnieje istotna różnica (bez wskazania kierunku) w Ocenie końcowej pomiędzy uczniami i uczennicami. Przyjmij significance level (alpha) na poziomie 0.05. Czy uda się odrzucić hipotezę zerową?

In [ ]:
path = './drive/MyDrive/PAD07'

import pandas as pd
import plotly.express as px
import scipy.stats

df = pd.read_csv(f"{path}/wyniki.csv", sep=',')
df

,plec,ocena_koncowa
0,F,6
1,F,6
2,F,10
3,F,15
4,F,10
...,...,...
352,M,9
353,M,16
354,M,7
355,M,10


In [ ]:
df.groupby('plec')['ocena_koncowa'].describe()

,count,mean,std,min,25%,50%,75%,max
plec,,,,,,,,
F,185.0,11.205405,3.174452,4.0,9.0,11.0,13.0,19.0
M,172.0,11.866279,3.258748,5.0,10.0,12.0,14.0,20.0


In [ ]:
px.box(df, x = 'plec', y = 'ocena_koncowa')

In [ ]:
scipy.stats.levene(df[df['plec'] == 'M']['ocena_koncowa'],
                   df[df['plec'] == 'F']['ocena_koncowa'])

LeveneResult(statistic=0.6144552033049334, pvalue=0.4336379490863752)

In [ ]:
scipy.stats.ttest_ind(
    df[df['plec'] == 'M']['ocena_koncowa'],
    df[df['plec'] == 'F']['ocena_koncowa'],
    equal_var = True,
    alternative = 'two-sided'
)

Ttest_indResult(statistic=1.9404771273257526, pvalue=0.053113174845791505)

hipoteza zerowa nie została odrzucona, p>0.05

Zadanie 2: Korzystając z danych w pliku ZyskiFirmyX.csv stwórz model regresji liniowej i biblioteki statmodels:
Wyświetl wykres pokazujący zyski w czasie. 
Czy da się zaobserwować trend? Jaki?
Wyświetl podsumowanie modelu.


In [4]:
import pandas as pd
import plotly.express as px
import scipy.stats as stats
import numpy as np
import statsmodels.formula.api as smf

path = './drive/MyDrive/PAD07'
df = pd.read_csv(f"{path}/ZyskiFirmyX.csv", sep=',')

df.rename(columns = {'Income (-500-2200)':'Rok', 'Annual Income (k$)':'Zysk'}, inplace = True)
model = smf.ols(formula="Zysk ~ Rok", data=df).fit()
print('P Values: ', model.pvalues.values)
print('Coef: ', model.params.values)
print("Std Errs", model.bse.values)
fig = px.scatter(df, x='Rok', y='Zysk',title = "Income over years", trendline="ols", trendline_color_override = 'red').show()
print(model.summary())

#Widzimy trend delilatnego spadku zysku na przestrzeni lat 
#podsumowanie modelu:
model = smf.ols(formula="Zysk ~ Rok", data=df).fit()
print(model.summary())

print("Model P Values:", model.pvalues.values)
print("Model Coef:", model.params.values)
print("Model Std Errs:", model.bse.values)

P Values:  [0.23400184 0.76000762]
Coef:  [ 1.37387953e+03 -1.83228081e-01]
Std Errs [1.15088837e+03 5.98994365e-01]


                            OLS Regression Results                            
Dep. Variable:                   Zysk   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                   0.09357
Date:                Sat, 28 Jan 2023   Prob (F-statistic):              0.760
Time:                        09:53:23   Log-Likelihood:                -1521.3
No. Observations:                 200   AIC:                             3047.
Df Residuals:                     198   BIC:                             3053.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1373.8795   1150.888      1.194      0.2

Zadanie 3: Korzystając z danych w pliku spendingscores.csv i biblioteki statmodels:
stwórz model regresji liniowej, gdzie zmienną zależną jest Spending score, a zmiennymi niezależnymi pozostałe zmienne.
Z modelu wyświetl:
p values
standard dev
coefficients
Sprawdź czy istnieje korelacja pomiędzy zmiennymi. Pokaż to na wykresie, np. z biblioteki plotly express.
Korzystając z eliminacji wstecznej usuń najmniej istotną zmienną i jeszcze raz wyświetl model.
UWAGA: zwróć uwagę na zmienne będące kategoriami!


In [21]:
import pandas as pd
import plotly.express as px
import scipy.stats as stats
import numpy as np
import statsmodels.formula.api as smf

path = './drive/MyDrive/PAD07'

df = pd.read_csv(f"{path}/spendingscores.csv")
df.head()

,Gender,Age,AnnualIncome,SpendingScore
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40


In [22]:
df['Gender'] =  df.apply(lambda x: 1 if x['Gender'] == 'Male' else 0 if x['Gender'] == 'Female' else None, axis = 1)
set(df['Gender'])

{0, 1}

In [23]:
px.scatter(df, "Age", "SpendingScore", "Gender" , title = "Spending Score with Age and Gender", trendline="ols").show()
px.scatter(df, "Age", "AnnualIncome", "Gender",  title = "Annual Income with Age and Gender", trendline="ols").show()
px.scatter(df, "AnnualIncome", "SpendingScore" , "Age",  title = "Spending Score with Age and Annual Income", trendline="ols").show()
px.scatter(df, "AnnualIncome","SpendingScore",  "Gender",  title = "Spending Score with Gender and Annual Income", trendline="ols").show()

In [25]:
model = smf.ols(formula="SpendingScore ~ Gender + Age + AnnualIncome", data=df).fit()
print(model.summary())

print('Tabela korelacji zmiennych')
df.corr()

print("Model P Values:", model.pvalues.values)
print("Model Std Errs:", model.bse.values)
print("Model Coef:", model.params.values)

                            OLS Regression Results                            
Dep. Variable:          SpendingScore   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     7.960
Date:                Sat, 28 Jan 2023   Prob (F-statistic):           4.91e-05
Time:                        10:20:48   Log-Likelihood:                -922.05
No. Observations:                 200   AIC:                             1852.
Df Residuals:                     196   BIC:                             1865.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       73.9300      6.642     11.130   

In [26]:
# Najbardziej skorelowane zmienne to Age i Gender.
model1 = smf.ols(formula="Gender ~ Age", data=df).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                 Gender   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.7363
Date:                Sat, 28 Jan 2023   Prob (F-statistic):              0.392
Time:                        10:21:33   Log-Likelihood:                -143.34
No. Observations:                 200   AIC:                             290.7
Df Residuals:                     198   BIC:                             297.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3558      0.104      3.411      0.0

In [27]:
# Podsumowanie modelu po zrezygnowaniu ze zmiennej AnnualIncome
model2 = smf.ols(formula="SpendingScore ~ Gender + Age", data=df).fit()
print(model2.summary())

print("Model P Values:", model2.pvalues.values)
print("Model Std Errs:", model2.bse.values)
print("Model Coef:", model2.params.values)

                            OLS Regression Results                            
Dep. Variable:          SpendingScore   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     11.99
Date:                Sat, 28 Jan 2023   Prob (F-statistic):           1.22e-05
Time:                        10:21:57   Log-Likelihood:                -922.05
No. Observations:                 200   AIC:                             1850.
Df Residuals:                     197   BIC:                             1860.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.4089      5.281     14.089      0.0